In [16]:
import io
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

html = requests.get('http://bechdeltest.com/api/v1/getAllMovies').content
df = pd.read_json(io.StringIO(html.decode('utf-8')))

In [8]:
df.head()

,rating,imdbid,id,title,year
0,0,3155794,9602,Passage de Venus,1874
1,0,14495706,9804,La Rosace Magique,1877
2,0,2221420,9603,Sallie Gardner at a Gallop,1878
3,0,12592084,9806,Le singe musicien,1878
4,0,7816420,9816,Athlete Swinging a Pick,1881


In [12]:
df.to_csv('/home/jdtganding/Documents/data-engineering-zoomcamp/week_7_PROJECT/data/BechdelTestMovies.csv', index=None)

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By

options = webdriver.FirefoxOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

driver = webdriver.Firefox(options=options)
driver.get("https://awardsdatabase.oscars.org/") 

#select award categories
driver.find_element(By.XPATH,"//button[contains(@class,'awards-basicsrch-awardcategory')]").click()
driver.find_element(By.XPATH,"//b[contains(text(),'Current Categories')]").click()

#select starting award year
driver.find_element(By.XPATH,"//button[contains(@class,'awards-advsrch-yearsfrom')]").click()
driver.find_element(By.XPATH,"//div[@class='btn-group multiselect-btn-group open']//input[@value='1']").click()

#select ending award year
driver.find_element(By.XPATH,"//button[contains(@class,'awards-advsrch-yearsto')]").click()
year_latest = len(driver.find_elements(By.XPATH,"//div[@class='btn-group multiselect-btn-group open']//li"))-2
driver.find_element(By.XPATH,f"//div[@class='btn-group multiselect-btn-group open']//input[@value='{year_latest}']").click()

#search to view results
driver.find_element(By.XPATH,'//*[@id="btnbasicsearch"]').click()

#wait for all results to show
time.sleep(60)

#get html source for BeautifulSoup extraction
page_source = driver.page_source

#close driver
driver.close()

In [18]:
soup = BeautifulSoup(page_source, "lxml")
results_container = soup.find('div', {'id':'resultscontainer'})

with open ("sample.txt", "w") as file:
    file.write(str(results_container))

In [23]:
award_group = results_container.find('div', class_='awards-result-chron result-group group-awardcategory-chron')
award_group.find('div', class_='result-group-title').find('a').text

'1927/28 (1st)'